## Scenario 2: A cross-funcional team with one data scientist working on an ML model

MLFlow Setup:
* tracking server: yes, local server
* backend store: sqllite database
* artifacts store: local filesystem

The experiment can be explored locally by acessing the local tracking server

To run this exemple you need to launch the mlflow server locally by running the following command in your terminal:

**mlflow server --backend-store-uri sqlite:///backend.db**

In [4]:
import mlflow

# sets the tracking server URI of MLflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [5]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1737223658536, experiment_id='0', last_update_time=1737223658536, lifecycle_stage='active', name='Default', tags={}>]

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/01/18 18:09:39 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/01/18 18:09:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: 'mlflow-artifacts:/1/bae7314c65a14669b070a7f70d190868/artifacts'
🏃 View run unique-grub-135 at: http://127.0.0.1:5000/#/experiments/1/runs/bae7314c65a14669b070a7f70d190868
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [8]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1737223779193, experiment_id='1', last_update_time=1737223779193, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1737223658536, experiment_id='0', last_update_time=1737223658536, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model regstry

In [9]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [11]:
#check registered model
client.search_registered_models()

[]

In [15]:
#register a model with name 'iris-classifier' with ref: run_id
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/01/18 18:18:19 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1737224298999, current_stage='None', description='', last_updated_timestamp=1737224298999, name='iris-classifier', run_id='bae7314c65a14669b070a7f70d190868', run_link='', source='mlflow-artifacts:/1/bae7314c65a14669b070a7f70d190868/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>